This is a combined retriever / hybrid retriever 

## Ensemble Retriever Doesn't seem to work

In [1]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.retrievers import BM25Retriever
#creating my own hybrid retriever
from langchain_core.documents import Document

/Users/maruthienugula/RAG_learnings/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Step 1: Sample documents
docs = [
    Document(page_content="LangChain helps build LLM applications."),
    Document(page_content="Pinecone is a vector database for semantic search."),
    Document(page_content="The Eiffel Tower is located in Paris."),
    Document(page_content="Langchain can be used to develop agentic ai application."),
    Document(page_content="Langchain has many types of retrievers.")
]

# Step 2: Dense Retriever (FAISS + HuggingFace)
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
dense_vectorstore = FAISS.from_documents(docs, embedding_model)
dense_retriever = dense_vectorstore.as_retriever()

In [3]:
### Sparse Retriever(BM25)
sparse_retriever=BM25Retriever.from_documents(docs)
sparse_retriever.k=3 ##top- k documents to retriever

In [5]:
# without ensemble retriever
query = "How can I build an application using LLMs?"
results = dense_retriever.invoke(query)

# Step 6: Print results
for i, doc in enumerate(results):
    print(f"\n🔹 Document {i+1}:\n{doc.page_content}")


🔹 Document 1:
LangChain helps build LLM applications.

🔹 Document 2:
Langchain can be used to develop agentic ai application.

🔹 Document 3:
Pinecone is a vector database for semantic search.

🔹 Document 4:
Langchain has many types of retrievers.


In [14]:
from langchain_core.callbacks import CallbackManagerForRetrieverRun


In [ ]:
from collections import defaultdict
from typing import List
from langchain_core.documents import Document

class HybridRetriever:
    def __init__(self, dense_retriever, sparse_retriever, w_dense=0.7, w_sparse=0.3):
        self.dense = dense_retriever
        self.sparse = sparse_retriever
        self.w_dense = w_dense
        self.w_sparse = w_sparse

    def invoke(self, query: str, k: int = 5) -> List[Document]:
        dense_docs = self.dense._get_relevant_documents(query,run_manager=CallbackManagerForRetrieverRun.get_noop_manager())
        sparse_docs = self.sparse._get_relevant_documents(query,run_manager=CallbackManagerForRetrieverRun.get_noop_manager())

        scores = defaultdict(float)
        doc_map = {}

        # Assign dense scores (inverse rank as proxy)
        for rank, doc in enumerate(dense_docs):
            doc_id = doc.page_content
            scores[doc_id] += self.w_dense * (1 / (rank + 1))
            doc_map[doc_id] = doc

        # Assign sparse scores
        for rank, doc in enumerate(sparse_docs):
            doc_id = doc.page_content
            scores[doc_id] += self.w_sparse * (1 / (rank + 1))
            doc_map[doc_id] = doc

        # Sort by combined score
        ranked_docs = sorted(scores.items(), key=lambda x: x[1], reverse=True)

        return [doc_map[doc_id] for doc_id, _ in ranked_docs[:k]]


In [20]:
## step 4 : Combine with Ensemble Retriever
hybrid_retriever=HybridRetriever(
    dense_retriever=dense_retriever,
    sparse_retriever=sparse_retriever # 0.7 - dense default
)

In [21]:
hybrid_retriever

In [23]:
# Step 5: Query and get results
query = "How can I build an application using LLMs?"
results = hybrid_retriever.invoke(query)

# Step 6: Print results
for i, doc in enumerate(results):
    print(f"\n🔹 Document {i+1}:\n{doc.page_content}")


🔹 Document 1:
LangChain helps build LLM applications.

🔹 Document 2:
Langchain can be used to develop agentic ai application.

🔹 Document 3:
Langchain has many types of retrievers.

🔹 Document 4:
Pinecone is a vector database for semantic search.


## RAG Pipeline with Hybrid Retriever

In [25]:
from langchain.chat_models import init_chat_model
from langchain_core.prompts import PromptTemplate


In [26]:
prompt = PromptTemplate.from_template("""
Answer the question based on the below context
Context : {context},
question : {input}
""")

llm = init_chat_model(model="groq:llama-3.3-70b-versatile")
llm

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 32768, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x128088320>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x129654530>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [45]:
from langchain_core.output_parsers import StrOutputParser

In [44]:
from langchain_core.runnables import RunnableLambda,RunnablePassthrough

def retrieve_docs(query):
    return hybrid_retriever.invoke(query=query)


In [51]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

def retrieve_docs(query):
    return hybrid_retriever.invoke(query)

rag_chain = (
    {
        "docs": RunnableLambda(retrieve_docs),
        "input": RunnablePassthrough(),
    }
    | RunnableLambda(
        lambda x: {
            "context": "\n\n".join(doc.page_content for doc in x["docs"]),
            "docs": x["docs"],
            "input": x["input"],
        }
    )
    | RunnableLambda(
        lambda x: {
            "answer": (
                prompt
                | llm
                | StrOutputParser()
            ).invoke(
                {"context": x["context"], "input": x["input"]}
            ),
            "context": x["docs"],
        }
    )
)


In [52]:
response = rag_chain.invoke("How can I build an app using LLMs?")


In [53]:
#  Ask a question
query =  "How can I build an app using LLMs?"
response = rag_chain.invoke(query)

#  Output
print(" Answer:\n", response)

print("\n Source Documents:")
for i, doc in enumerate(response["context"]):
    print(f"\nDoc {i+1}: {doc.page_content}")

 Answer:
 {'answer': 'You can use LangChain to build an application using Large Language Models (LLMs). LangChain is a framework that helps developers build LLM applications, including agentic AI applications, and provides various tools and features to support the development process.', 'context': [Document(metadata={}, page_content='LangChain helps build LLM applications.'), Document(metadata={}, page_content='Langchain can be used to develop agentic ai application.'), Document(metadata={}, page_content='Langchain has many types of retrievers.'), Document(id='9c48d1af-d932-4123-a9e7-5fc748ae4881', metadata={}, page_content='Pinecone is a vector database for semantic search.')]}

 Source Documents:

Doc 1: LangChain helps build LLM applications.

Doc 2: Langchain can be used to develop agentic ai application.

Doc 3: Langchain has many types of retrievers.

Doc 4: Pinecone is a vector database for semantic search.


Thought this would run this way but hell its deprecated and doesn't

In [54]:
query = "What is your Age ?"
docs = hybrid_retriever.invoke(query)

context = "\n\n".join(doc.page_content for doc in docs)
answer = llm.invoke(prompt.format(context = context,input=query))

response = {
    "answer":answer,
    "context":docs
}

In [56]:
#  Ask a question
query = {"input": "How can I build an app using LLMs?"}
response = rag_chain.invoke(query)

#  Output
print(" Answer:\n", response["answer"])

print("\n Source Documents:")
for i, doc in enumerate(response["context"]):
    print(f"\nDoc {i+1}: {doc.page_content}")

AttributeError: 'dict' object has no attribute 'replace'